In [2]:
import os
from PIL import Image
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from tqdm import tqdm

# Define the path to the directory
directory_path = r"C:\Users\shaif\Downloads\Compressed\archive\chest_xray\train"

# Initialize empty lists for images and labels
X_train = []
Y_train = []
class_label = 0

# Loop through subdirectories (classes)
for class_folder in tqdm(sorted(os.listdir(directory_path))):
    class_path = os.path.join(directory_path, class_folder)
    
    for image_file in os.listdir(class_path):
        if image_file.endswith('.jpeg'):
            image_path = os.path.join(class_path, image_file)
            
            # Load image, convert to RGB and resize
            img = Image.open(image_path).convert('RGB')
            img = img.resize((32, 32))
            img_array = np.array(img)
            
            # Append image and label to lists
            X_train.append(img_array)
            Y_train.append(class_label)
            
    class_label = class_label + 1

    
# Convert lists to numpy arrays
X_train = np.array(X_train)
Y_train = np.array(Y_train)

print(f'X_train shape: {X_train.shape}')
print(f'Y_train shape: {Y_train.shape}')

100%|███████████████████████████████████████████████████████████████| 2/2 [01:40<00:00, 50.41s/it]

X_train shape: (5216, 32, 32, 3)
Y_train shape: (5216,)


In [3]:
import os
from PIL import Image
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from tqdm import tqdm

# Define the path to the directory
directory_path = r"C:\Users\shaif\Downloads\Compressed\archive\chest_xray\val"

# Initialize empty lists for images and labels
X_val = []
Y_val = []
class_label = 0

# Loop through subdirectories (classes)
for class_folder in tqdm(sorted(os.listdir(directory_path))):
    class_path = os.path.join(directory_path, class_folder)
    
    for image_file in os.listdir(class_path):
        if image_file.endswith('.jpeg'):
            image_path = os.path.join(class_path, image_file)
            
            # Load image, convert to RGB and resize
            img = Image.open(image_path).convert('RGB')
            img = img.resize((32, 32))
            img_array = np.array(img)
            
            # Append image and label to lists
            X_val.append(img_array)
            Y_val.append(class_label)
            
    class_label = class_label + 1

# Convert lists to numpy arrays
X_val = np.array(X_val)
Y_val = np.array(Y_val)

print(f'X_train shape: {X_val.shape}')
print(f'Y_train shape: {Y_val.shape}')

100%|███████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  4.69it/s]

X_train shape: (16, 32, 32, 3)
Y_train shape: (16,)


In [5]:
import tensorflow as tf
#tf.keras.backend.clear_session()
from tensorflow.keras.utils import to_categorical
YT = to_categorical(Y_train)
VT = to_categorical(Y_val)

In [4]:
Y_val

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1])

In [17]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.resnet import ResNet50, preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
num_classes = 2
# Create the ResNet model without the top (fully connected) layers
base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(32, 32, 3))    
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(num_classes, activation='softmax'))  # Change units to match the number of classes
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
epochs = 50
model.fit(X_train,YT, epochs=epochs,batch_size=32, validation_split = .1)

Epoch 1/50
147/147 [==============================] - 7s 31ms/step - loss: 0.2169 - accuracy: 0.9254 - val_loss: 0.3388 - val_accuracy: 0.9502
Epoch 2/50
147/147 [==============================] - 4s 29ms/step - loss: 0.1216 - accuracy: 0.9585 - val_loss: 0.1658 - val_accuracy: 0.9444
Epoch 3/50
147/147 [==============================] - 4s 29ms/step - loss: 0.0708 - accuracy: 0.9783 - val_loss: 0.1173 - val_accuracy: 0.9540
Epoch 4/50
147/147 [==============================] - 4s 29ms/step - loss: 0.0632 - accuracy: 0.9785 - val_loss: 0.0771 - val_accuracy: 0.9713
Epoch 5/50
147/147 [==============================] - 4s 29ms/step - loss: 0.0430 - accuracy: 0.9851 - val_loss: 0.0514 - val_accuracy: 0.9885
Epoch 6/50
147/147 [==============================] - 4s 29ms/step - loss: 0.0594 - accuracy: 0.9821 - val_loss: 0.0333 - val_accuracy: 0.9904
Epoch 7/50
147/147 [==============================] - 4s 29ms/step - loss: 0.0398 - accuracy: 0.9872 - val_loss: 0.0374 - val_accuracy: 0.9885

In [15]:
model.fit(X_train,YT, epochs=2,batch_size=32, validation_split = .1)

Epoch 1/2
147/147 [==============================] - 5s 31ms/step - loss: 0.0252 - accuracy: 0.9906 - val_loss: 0.1588 - val_accuracy: 0.9617
Epoch 2/2
147/147 [==============================] - 4s 28ms/step - loss: 0.0447 - accuracy: 0.9834 - val_loss: 0.0130 - val_accuracy: 0.9923


In [16]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

# Make predictions on the test dataset
predictions = model.predict(X_val)

# Convert one-hot encoded predictions back to class labels
predicted_labels = np.argmax(predictions, axis=1)

# Calculate accuracy
accuracy = accuracy_score(Y_val, predicted_labels)

# Calculate F1 score
f1 = f1_score(Y_val, predicted_labels, average='weighted')

print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")

Accuracy: 0.5
F1 Score: 0.3333333333333333


In [ ]:
print(".......................................................................................")
print(".......................................................................................")

In [8]:
import tensorflow
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.resnet import ResNet50, preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
num_classes = 2
model = tensorflow.keras.models.load_model(r"D:\che_model.h5")

x = model.layers[-4].output  # Access the last 4th layer from the end
x = GlobalAveragePooling2D(keepdims=False)(x)  # Explicitly specify keepdims
output = Dense(num_classes, activation='softmax')(x)
new_model = tensorflow.keras.models.Model(inputs=model.input, outputs=output) 
new_model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
epochs = 50
new_model.fit(X_train,YT, epochs=epochs,batch_size=32, validation_split = .1)

Epoch 1/50
147/147 [==============================] - 25s 63ms/step - loss: 0.2405 - accuracy: 0.9427 - val_loss: 0.0372 - val_accuracy: 0.9885
Epoch 2/50
147/147 [==============================] - 7s 49ms/step - loss: 0.0944 - accuracy: 0.9672 - val_loss: 0.0948 - val_accuracy: 0.9751
Epoch 3/50
147/147 [==============================] - 7s 51ms/step - loss: 0.0538 - accuracy: 0.9781 - val_loss: 0.0758 - val_accuracy: 0.9770
Epoch 4/50
147/147 [==============================] - 7s 47ms/step - loss: 0.0475 - accuracy: 0.9825 - val_loss: 0.0547 - val_accuracy: 0.9828
Epoch 5/50
147/147 [==============================] - 7s 50ms/step - loss: 0.0364 - accuracy: 0.9862 - val_loss: 0.1311 - val_accuracy: 0.9713
Epoch 6/50
147/147 [==============================] - 7s 47ms/step - loss: 0.0329 - accuracy: 0.9874 - val_loss: 0.2459 - val_accuracy: 0.9502
Epoch 7/50
147/147 [==============================] - 7s 45ms/step - loss: 0.0501 - accuracy: 0.9830 - val_loss: 0.0414 - val_accuracy: 0.988

In [ ]:
new_model.fit(X_train,YT, epochs=2,batch_size=32, validation_split = .1)

In [18]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
# Make predictions on the test dataset
predictions = new_model.predict(X_val)
# Convert one-hot encoded predictions back to class labels
predicted_labels = np.argmax(predictions, axis=1)
# Calculate accuracy
accuracy = accuracy_score(Y_val, predicted_labels)
# Calculate F1 score
f1 = f1_score(Y_val, predicted_labels, average='weighted')
print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")

Accuracy: 0.9375
F1 Score: 0.9372549019607843


In [ ]:
print(".......................................................................................")
print(".......................................................................................")

In [8]:
from tensorflow import keras
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
    
num_classes = 10  
model = keras.models.load_model(r"C:\Users\shaif\Downloads\imb_ssl.h5")
x = model.layers[-8].output  
output = Dense(num_classes, activation='softmax')(x)
new_model = keras.models.Model(inputs=model.input, outputs=output)
new_model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
new_model.fit(X_train, YT, batch_size=64, epochs=50, validation_split = .2)

Epoch 1/50
363/363 [==============================] - 42s 68ms/step - loss: 516.2614 - accuracy: 0.1743 - val_loss: 4347.4189 - val_accuracy: 0.0000e+00
Epoch 2/50
363/363 [==============================] - 21s 58ms/step - loss: 193.0619 - accuracy: 0.1802 - val_loss: 2157.3953 - val_accuracy: 0.0109
Epoch 3/50
363/363 [==============================] - 22s 61ms/step - loss: 62.5039 - accuracy: 0.1754 - val_loss: 1404.2903 - val_accuracy: 0.1643
Epoch 4/50
363/363 [==============================] - 23s 63ms/step - loss: 34.9450 - accuracy: 0.1793 - val_loss: 934.8641 - val_accuracy: 0.0000e+00
Epoch 5/50
363/363 [==============================] - 20s 56ms/step - loss: 20.3900 - accuracy: 0.1827 - val_loss: 715.6703 - val_accuracy: 0.0000e+00
Epoch 6/50
363/363 [==============================] - 22s 61ms/step - loss: 15.4835 - accuracy: 0.1938 - val_loss: 586.2610 - val_accuracy: 0.0000e+00
Epoch 7/50
363/363 [==============================] - 23s 65ms/step - loss: 12.3891 - accuracy: 0

In [9]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

# Assuming you have trained a model and obtained predicted probabilities on x_test
y_pred_prob = new_model.predict(V_val)

# Convert predicted probabilities to predicted labels
y_pred = np.argmax(y_pred_prob, axis=1)

# Convert y_test to predicted labels format
y_test_labels = np.argmax(y_val, axis=1)

# Calculate accuracy
accuracy = accuracy_score(y_test_labels, y_pred)
print("Accuracy:", accuracy)

# Calculate F1 score (micro-average)
f1_micro = f1_score(y_test_labels, y_pred, average='micro')
print("F1 Score (Micro):", f1_micro)

NameError: name 'V_val' is not defined

In [ ]:
print(".......................................................................................")
print(".......................................................................................")

In [ ]:
print(".......................................................................................")
print(".......................................................................................")